### Visualizing ephys data and saving out nwb files
Using the NeuroConv environment bc I changed a method in neo

In [ ]:
from decode_lab_code.wrangle import nwbtools
from decode_lab_code.preprocessing import signal_utils
import matplotlib.pyplot as plt
import numpy as np

### Instantiate the read_nlx object in the converters file
First we need to define our directory for data ("folder_path")
Next, we can instantiate the read_nlx object for the purposes of working with and loading our data


In [ ]:
#folder_path = '/Users/js0403/local data/2021-03-31_08-59-02 16eB R1 10min rec after sec drive cells on 2 and 3 - Control'
folder_path = '/Users/js0403/local data/2020-06-26_16-56-10 9&10eb male ACTH ELS'
data_object = nwbtools.read_nlx(folder_path)

##### Now we can read our ephys data
If clusters are detected, the code will report to user

In [ ]:
data_object.read_ephys()

##### Read the history attribute to see your variables
history reports on available attributes (which can typically be seen by adding a period next to your object), followed by a description as such

folder_path (attribute name): directory of the dataset (definition)

In [ ]:
data_object.history

### Using the .history attribute, lets access some data
.tt_data is described in .history above

We can also access the dictionary keys to see what options are available

In [ ]:
print("here are the csc names:",data_object.csc_data_names)
print("here is how you access the first element:", data_object.csc_data_names[0])

#### Here, we can use ._names to access information from ._data attributes

In [ ]:
csc1_signal = data_object.csc_data[data_object.csc_data_names[0]]
fs = data_object.csc_data_fs
print("Here is LFP from",data_object.csc_data_names[0],csc1_signal)

time_x = np.linspace(0,1,fs)
plt.plot(time_x[0:fs],csc1_signal[0:fs])
plt.title("Example signal obtained from the read_nlx class")

#### The process_signal class can perform operations on LFP signals, but also let us visualize data quickly
Here is an example of rereferencing

In [ ]:
sig = signal_utils.process_signal(data_object.csc_data, fs = fs)
sig.rereference()

### The signal_utils file contains helper functions for plotting
This should probably be its own class or file

In [ ]:
colors = ['k','k','k','k','r','r','r','r','b','b','b','b','m','m','m','m']
signal_utils.multi_plotter(data = sig.signal_rereferenced, fs=fs, time_range = [1, 1.5], color = colors)

### Now lets use the nwb_utils object to store this data and save out as an NWB file
I'm also going to add an nwb2nlx function that reads the NWB file, then organizes the LFP and spiking data back into dictionaries, which can be run through signal_utils

Signal_utils will also use pynapple for analysis

### Using the signal_utils objects

In [ ]:
# TODO: implement signal_utils

# run a filter
sig.butterworth_filter(lowcut = 4, highcut = 12, fs = 32000)

# rereference and plot new results
sig.rereference(rereference_mode = 'CAR')
sig.butterworth_filter(data = sig.signal_rereferenced,lowcut = 4,highcut = 12,fs = 32000)

plt.subplot(3,1,1)
plt.plot(sig.data['TT1a.ncs'][0:32000],'k',linewidth=1)
plt.plot(sig.signal_filtered['TT1a.ncs'][0:32000],'r',linewidth=2)
plt.subplot(3,1,2)
plt.plot(sig.signal_rereferenced['TT1a.ncs'][0:32000],'k',linewidth=1)
plt.plot(sig.signal_filtered['TT1a.ncs'][0:32000],'r',linewidth=2)
plt.subplot(3,1,3)
plt.plot(sig.data['TT1a.ncs'][0:32000],'k',linewidth=1)
plt.plot(sig.signal_rereferenced['TT1a.ncs'][0:32000],'r',linewidth=1)
    

### Between signal correlation

In [ ]:
#%
# testing correlation after signal rereferencing
import pandas as pd
import matplotlib.pyplot as plt
import os

data_pd = pd.DataFrame.from_dict(sig.data)
data_pd_ref = pd.DataFrame.from_dict(sig.signal_rereferenced)

# pairwise correlation
data_cor = pd.DataFrame.corr(self=data_pd, method = 'pearson', min_periods=1, numeric_only = False)
data_cor_reref = pd.DataFrame.corr(self=data_pd_ref, method = 'pearson', min_periods=1, numeric_only = False)

fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2)
pos1 = ax1.imshow(data_cor)
ax1.set_xticks(range(len(data_cor)), data_cor.columns, rotation='vertical')
ax1.set_yticks(range(len(data_cor)), data_cor.index)
#ax1.yaxis('off')
fig.colorbar(pos1, ax=ax1, shrink=0.5)

pos2 = ax2.imshow(data_cor_reref)
ax2.set_xticks(range(len(data_cor)), data_cor.columns, rotation='vertical')
ax2.set_yticks(range(len(data_cor)), data_cor.index, color='w')
fig.colorbar(pos2, ax=ax2, shrink=0.5)
fig.savefig(folder_path, format='eps')